In [ ]:
from bs4 import BeautifulSoup
import pandas as pd 
import seaborn as sns
import requests
import re


In [ ]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36'}
title_page = requests.get('https://www.imdb.com/chart/top/?ref_=nv_mv_250', headers=headers)
tp_soup = BeautifulSoup(title_page.content, 'html.parser')

In [ ]:
top_movies = tp_soup.find_all('li', attrs={'class': 'ipc-metadata-list-summary-item sc-1364e729-0 caNpAE cli-parent'})
titles = [re.sub(r'\d+\.\s*','' ,str(item.find('h3', attrs={'class':'ipc-title__text'}).contents[0])) for item in top_movies]
display(titles)


In [ ]:
user_rating = tp_soup.find_all('span', attrs={'class':'ipc-rating-star ipc-rating-star--base ipc-rating-star--imdb ratingGroup--imdb-rating'})
user_rating_comb = [item.get_text(strip=True ) for item in user_rating]
user_rating = [re.sub(r'\(.+\)', '', item) for item in user_rating_comb]
user_rating

In [ ]:
rating_count = [re.search(r'\(\d+(\.\d)?\w', item).group() for item in user_rating_comb]
rating_count = [re.sub(r'\(', '', item) for item in rating_count]
# display(rating_count)
for i in range(len(rating_count)):

    if rating_count[i][-1] == 'M':

        rating_count[i] = float(rating_count[i][:-1])*1000000
    elif rating_count[i][-1] == "K":

        rating_count[i] = float(rating_count[i][:-1])*1000
rating_count

len(rating_count)

First Analysis: Year the movies came out.

Note: The first movie with synchronized dialogue was released in 1927

Second Analysis: Place in table vs number of ratings

In [ ]:
place = [i for i in range(1, 251)]
movie_df = pd.DataFrame({'Title':titles, 'Place': place, 'Number_of_ratings': rating_count, 'Average_rating':user_rating})
movie_df['Average_rating'] = pd.to_numeric(movie_df['Average_rating'])

In [ ]:
movie_df['Place'][10:]

In [ ]:
import scipy.stats as stats
import matplotlib.pyplot as plt

sns.scatterplot(data=movie_df, x='Place', y='Number_of_ratings')
print('all movies: ', stats.pearsonr(x=movie_df['Place'], y=movie_df['Number_of_ratings']))
plt.figure()
smaller_movie_df = movie_df.iloc[:25]
sns.scatterplot(data=smaller_movie_df, x='Place', y='Number_of_ratings')
stats.pearsonr(x=smaller_movie_df['Place'], y=smaller_movie_df['Number_of_ratings'])

In [ ]:
sns.scatterplot(data=movie_df, x='Average_rating', y='Number_of_ratings')
print('Rating vs number of ratings: ', stats.pearsonr(x=movie_df['Average_rating'], y=movie_df['Number_of_ratings']))
plt.figure()